In [101]:
import os, sys

In [102]:
'SUMO_HOME' in os.environ

True

In [103]:
tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
sys.path.append(tools)

In [104]:
sumoBinary = "C:/Program Files (x86)/DLR/Sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "C:/Users/sreeniva/Desktop/Reinforcement Learning/madrl_traffic_control/Sumo Stuff/hello.sumocfg"]

In [105]:
import traci

In [106]:
traci.start(sumoCmd)

(15, 'SUMO 0.30.0')

In [107]:
step=0
while step < 10:
   traci.simulationStep()
   step += 1

In [88]:
traci.trafficlights.getIDList()

['2']

In [110]:
import traci
import math

CAR_WIDTH = 5
MAX_HEIGHT = 200 / CAR_WIDTH
MAX_LENGTH = 200 / CAR_WIDTH
lane_ids = ["left-right-1_0", "left-right-2_0",
            "right-left-1_0", "right-left-2_0",
            "up-down-1_0", "up-down-2_0",
            "down-up-1_0", "down-up-2_0", ]


def initialize_matrix():
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    dtse_map = [[(vehicle_present, vehicle_vel, vehicle_id)
                 for x in range(MAX_LENGTH)] for y in range(MAX_HEIGHT)]
    return dtse_map


def get_lane_dimensions():
    # horizontal lane
    lane_width = traci.lane.getShape(lane_ids[0])[0][
        1] - traci.lane.getShape(lane_ids[3])[0][1]
    horizontal_x_min = 0.0
    horizontal_y_min = traci.lane.getShape(lane_ids[0])[0][1]
    horizontal_x_max = 200.0
    horizontal_y_max = horizontal_y_min + (2 * lane_width)

    # vertical lane
    vertical_x_min = traci.lane.getShape(lane_ids[4])[0][0]
    vertical_y_min = 0.0
    vertical_x_max = vertical_x_min + (2 * lane_width)
    vertical_y_max = 200.0

    return {'horizontal_x_min': horizontal_x_min / CAR_WIDTH,
            'horizontal_x_max': horizontal_x_max / CAR_WIDTH,
            'horizontal_y_min': horizontal_y_min / CAR_WIDTH,
            'horizontal_y_max': horizontal_y_max / CAR_WIDTH,
            'vertical_x_min': vertical_x_min / CAR_WIDTH,
            'vertical_x_max': vertical_x_max / CAR_WIDTH,
            'vertical_y_min': vertical_y_min / CAR_WIDTH,
            'vertical_y_max': vertical_y_max / CAR_WIDTH}


def print_matrix(matrix):
    output_str = ""
    for i in range(0, MAX_LENGTH):
        for j in range(0, MAX_HEIGHT):
            output_str += str(matrix[i][j]) + "\t"
        output_str += "\n"
    print output_str


def adjust_matrix_for_lanes(matrix):
    lane_dimensions = get_lane_dimensions()
    for x in range(0, MAX_LENGTH):
        for y in range(0, MAX_HEIGHT):
            if x > lane_dimensions.get('vertical_x_min') and x < lane_dimensions.get('vertical_x_max')\
                    and y > lane_dimensions.get('vertical_y_min') and y < lane_dimensions.get('vertical_y_max'):
                matrix[x][y][0] = 0
            if x > lane_dimensions.get('horizontal_x_min') and x < lane_dimensions.get('horizontal_x_max')\
                    and y > lane_dimensions.get('horizontal_y_min') and y < lane_dimensions.get('horizontal_y_max'):
                matrix[x][y][0] = 0

    return matrix


def get_left_right_dtse(x_min, x_max, y):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(x_max - x_min) / CAR_WIDTH))
    dtse_map = [[vehicle_present, vehicle_vel, vehicle_id] for x in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if x_pos > x_min and x_pos < x_max and y_pos == y:
            # make sure blocks are equally spaced starting from the junction
            block = (n_blocks -1) - int((x_max - x_pos) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_right_left_dtse(x_min, x_max, y):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(x_max - x_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for x in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if x_pos > x_min and x_pos < x_max and y_pos == y:
            block = int((x_pos - x_min) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_up_down_dtse(y_min, y_max, x):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(y_max - y_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for y in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if y_pos > y_min and y_pos < y_max and x_pos == x:
            # make sure blocks are equally spaced starting from the junction
            block = int((y_pos - y_min) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_down_up_dtse(y_min, y_max, x):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(y_max - y_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for y in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if y_pos > y_min and y_pos < y_max and x_pos == x:
            # make sure blocks are equally spaced starting from the junction
            block = int((y_max - y_pos) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

In [111]:
for lane in lane_ids:
    print lane, traci.lane.getShape(lane)

left-right-1_0 [(0.0, 48.35), (45.25, 48.35)]
left-right-2_0 [(54.75, 48.35), (100.0, 48.35)]
right-left-1_0 [(100.0, 51.65), (54.75, 51.65)]
right-left-2_0 [(45.25, 51.65), (0.0, 51.65)]
up-down-1_0 [(48.35, 100.0), (48.35, 54.75)]
up-down-2_0 [(48.35, 45.25), (48.35, 0.0)]
down-up-1_0 [(51.65, 0.0), (51.65, 45.25)]
down-up-2_0 [(51.65, 54.75), (51.65, 100.0)]


In [112]:
# left-right-1
[(x_min, y), (x_max, y1)] = traci.lane.getShape('left-right-1_0')
lr_1_dtse = get_left_right_dtse(x_min, x_max, y)

In [113]:
lr_1_dtse

[[-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [1, 4.0, 'type1.0'],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999]]

In [13]:
x_min

0.0

In [114]:
# right-left-1
[(x_max, y), (x_min, y1)] = traci.lane.getShape('right-left-1_0')
rl_1_dtse = get_left_right_dtse(x_min, x_max, y)
rl_1_dtse

[[-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [1, 4.5, 'type3.0'],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999]]

In [115]:
# left-right-2
[(x_min, y), (x_max, y1)] = traci.lane.getShape('left-right-2_0')
lr_2_dtse = get_left_right_dtse(x_min, x_max, y)

In [116]:
lr_2_dtse

[[-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999]]

In [17]:
traci.lane.getShape('up-down-1_0')

[(48.35, 100.0), (48.35, 54.75)]

In [97]:
[(x, y_max), (x1, y_min)] = traci.lane.getShape('up-down-1_0')
get_up_down_dtse(y_min, y_max, x)

[(-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 [1, 4.5, 'type2.16'],
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 [1, 0.5, 'type2.17'],
 (-1, 0, -999),
 (-1, 0, -999)]

In [19]:
vehicle_ids = traci.vehicle.getIDList()

In [21]:
# up-down-2
[(x, y_max), (x1, y_min)] = traci.lane.getShape('up-down-2_0')
ud_2_dtse = get_up_down_dtse(y_min, y_max, x)
ud_2_dtse

[(-1, 0, -999),
 [1, 4.499999999999997, 'type2.2'],
 (-1, 0, -999),
 (-1, 0, -999),
 [1, 5.0, 'type2.1'],
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999)]

In [22]:
traci.lane.getShape('down-up-1_0')

[(51.65, 0.0), (51.65, 45.25)]

In [83]:
# down-up-1
[(x, y_min), (x1, y_max)] = traci.lane.getShape('down-up-1_0')
du_1_dtse = get_down_up_dtse(y_min, y_max, x)
du_1_dtse

[(-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 (-1, 0, -999),
 [1, 0.5, 'type4.5'],
 (-1, 0, -999),
 (-1, 0, -999)]

In [26]:
from collections import defaultdict
vehicle_wait_times = defaultdict(lambda: defaultdict(lambda: 0.0))
min_speed = 0.1
# call this at every step
def get_avg_waiting_time():
    avg_wait_time = 0.0
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        if traci.vehicle.getSpeed(vehicle_id) < 0.1:
            vehicle_wait_times[vehicle_id]['t_wait'] += 1
        else:
            vehicle_wait_times[vehicle_id]['t_move'] += 1

        vehicle_wait_times[vehicle_id]['avg_wait_time'] =\
            vehicle_wait_times[vehicle_id]['t_wait'] / (vehicle_wait_times[vehicle_id]['t_wait'] + vehicle_wait_times[vehicle_id]['t_move'])

        # makes sure we don't include avg wait time from vehicles no longer on the network
        avg_wait_time +=  vehicle_wait_times[vehicle_id]['avg_wait_time']
    avg_wait_time = avg_wait_time / len(vehicle_ids)

    return avg_wait_time

In [28]:
get_avg_waiting_time()

0.4117647058823529

In [29]:
vehicle_wait_times

defaultdict(<function __main__.<lambda>>,
            {'type1.8': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.1': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.2': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.3': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.4': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.5': defaultdict(<function __main__.<lambda>>,
                         {'avg_wait_time': 0.0, 't_move': 1.0, 't_wait': 0.0}),
             'type2.6': defaultdict(<function __main__.<lambda>>,
                

In [79]:
total_waiting_time = 0
total_moving_time = 0
def get_avg_waiting_time_v2():
    global total_waiting_time, total_moving_time
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        if traci.vehicle.getSpeed(vehicle_id) < 0.1:
            total_waiting_time += 1
        else:
            total_moving_time += 1
    
    avg_wait_time = 1.0 * total_waiting_time / (total_waiting_time + total_moving_time)
    return avg_wait_time

In [80]:
get_avg_waiting_time_v2()

0.4117647058823529

In [108]:
traci.trafficlights.getIDList()

['2']

In [109]:
traci.trafficlights.getPhase('2')

0